In [1]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, load_subject_non_downsampled, extract_labels, extract_action_interval, split_data, split_info, to_device, get_innerspeech
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject_non_downsampled(1)
data, description = get_innerspeech(data, description)
labels = extract_labels(description)
data_interval = extract_action_interval(data, hz = 1024)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data_interval, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

1024
yo
[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


# Deep CNN
Implementation of the deep CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepCNN(nn.Module):
    def __init__(self, hz, interval = "full", dropout = 0.5):
        super(DeepCNN, self).__init__()
        first_channels = 25
        second_channels = 50
        third_channels = 100
        fourth_channels = 200

        ### Block 1
        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 10), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = first_channels, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(first_channels, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Max pooling
        self.maxpool = nn.MaxPool2d(kernel_size = (1,3), stride = (1,3))

        ### Block 2
        self.conv2 = nn.Conv2d(in_channels = first_channels, out_channels = second_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm2 = nn.BatchNorm2d(second_channels, False)

        ### Block 3
        self.conv3 = nn.Conv2d(in_channels = second_channels, out_channels = third_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm3 = nn.BatchNorm2d(third_channels, False)

        ### Block 4
        self.conv4 = nn.Conv2d(in_channels = third_channels, out_channels = fourth_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm4 = nn.BatchNorm2d(fourth_channels, False)

        # Classifier
        if interval == "action":
            if hz == 254:
                self.classifier = nn.Linear(600,4, bias = False)
            elif hz == 1024:
                self.classifier = nn.Linear(5400,4, bias = False)
        elif interval == "full":
            if hz == 254:
                self.classifier = nn.Linear(1800,4, bias = False)
            elif hz == 1024:
                self.classifier = nn.Linear(10400,4, bias = False)
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        # Block 1
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm1(res)
        res = self.elu(res)
        res = self.maxpool(res)
        res = self.dropout(res)
        # Block 2
        res = self.conv2(res)
        res = self.batchnorm2(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 3
        res = self.conv3(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 4
        res = self.conv4(res)
        res = self.batchnorm4(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

In [5]:
network = DeepCNN(interval = "action", hz = 1024).float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.23417721518987342
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.5125391642252604 	 1.4288313388824463
1 	  1.1832196553548178 	 1.6402068138122559
2 	  0.9622668584187826 	 1.8683867454528809
3 	  0.8431697209676107 	 1.9547569751739502
4 	  0.682554817199707 	 2.004453659057617
5 	  0.5770423253377278 	 2.043234348297119
6 	  0.4683230717976888 	 2.1316511631011963
7 	  0.41487642923990886 	 2.260991096496582
8 	  0.34209117889404295 	 2.3088133335113525
9 	  0.2634807904561361 	 2.2944014072418213
10 	  0.23848748207092285 	 2.332322597503662
11 	  0.19249793688456218 	 2.3699100017547607
12 	  0.16090758641560873 	 2.429518699645996
13 	  0.14106640815734864 	 2.4940738677978516
14 	  0.1259350061416626 	 2.387023448944092
15 	  0.10305298964182535 	 2.3308000564575195
16 	  0.09561254183451334 	 2.3460500240325928
17 	  0.08189308643341064 	 2.318917989730835
18 	  0.07461902300516764 	 2.408930778503418
19 	  0.06600199937820435 	 2.500496864318847

In [6]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)


ACCURACY: 0.7278481012658228
ACCURACY: 0.36363636363636365
ACCURACY: 0.25
